In [2]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-30 15:57:32,592 - utils - NumExpr defaulting to 4 threads.


In [3]:
# define the path of the data stored
mockname = 'single_combined_lite'
downsample = 0.05
datapath = os.path.join(_data, mockname)

# read the numpy files
for file in glob.glob(os.path.join(datapath, 's*.numpy')):
    df = np.loadtxt(file)
    filename = file.split('/')[-1].split('.')[0]
    outfile = os.path.join(_data, mockname, filename)

    if downsample is not None:
        keep = np.random.randint(0, len(df), int(len(df) * downsample))
        print(filename, len(df), len(df[keep]))
        np.save(outfile, df[keep])
    else:
        np.save(outfile, df)


s0100000 22500 1125
s0600000 22500 1125
s0700000 22500 1125
s0500000 22500 1125
s0400000 22500 1125
s0200000 22500 1125
s0300000 22500 1125


In [4]:
# define the path of the data stored
# mockname = 'single_combined'
datapath = os.path.join(_data, mockname)

# read the numpy files
files = {}
bia_pos = {}
for file in glob.glob(os.path.join(datapath, 's*.npy')):
    df = np.load(file)
    filename = file.split('/')[-1]

    # sensor coordinates
    spos = os.path.join(datapath, 'scanner_pos.txt')
    scan = lad.laod_scan_pos(spos)
    id = [i.decode("utf-8") for i in scan['scan']]
    keep = np.array(id) == filename[:3]
    _, sx, sy, sz = scan[keep][0]

    if df.shape[0] > 0:
        files[filename] = df
        bia = np.full((len(df),3), np.array([sx, sy, sz]))
        bia_pos[filename] = bia
        print(filename, df.shape, bia.shape)
    else:
        print('file %s empty' %(filename))

    
    
# concatenate all data
df = np.concatenate(list(files.values()))
bias = np.concatenate(list(bia_pos.values()))
print(df.shape)
print(bias.shape)

s0200000.npy (1125, 16) (1125, 3)
s0300000.npy (1125, 16) (1125, 3)
s0700000.npy (1125, 16) (1125, 3)
s0600000.npy (1125, 16) (1125, 3)
s0500000.npy (1125, 16) (1125, 3)
s0100000.npy (1125, 16) (1125, 3)
s0400000.npy (1125, 16) (1125, 3)
(7875, 16)
(7875, 3)


In [5]:
# pass this to a pandas data frame for simplicity
scan = pd.DataFrame(df, columns=['timestamp', 'yaw', 'pitch', 'distance','distance_noise',
                                'x','y','z',
                                'x_noise','y_noise','z_noise',
                                'object_id', 'color0', 'color1','color2', 'idx'])

bia = pd.DataFrame(bias, columns=['sx', 'sy', 'sz'])

# concat
scan = pd.concat((scan, bia), axis=1)

In [6]:
scan

,timestamp,yaw,pitch,distance,distance_noise,x,y,z,x_noise,y_noise,z_noise,object_id,color0,color1,color2,idx,sx,sy,sz
0,0.0,3.107635,0.274372,14.245876,14.244353,0.465591,-3.798352,8.488060e+00,0.465541,-3.796876,8.488436,1.986094e+09,204.0,204.0,204.0,10333.0,0.0,10.0,12.0
1,0.0,3.370452,0.224252,11.695342,11.697829,-2.590151,-0.897733,8.636732e+00,-2.590701,-0.900050,8.636017,1.986094e+09,204.0,204.0,204.0,18572.0,0.0,10.0,12.0
2,0.0,3.402424,-0.219634,18.325695,18.324807,-4.619662,-3.057175,4.768372e-07,-4.619439,-3.056543,0.000581,1.851878e+09,204.0,204.0,204.0,19529.0,0.0,10.0,12.0
3,0.0,3.073756,-0.067837,21.571997,21.570236,1.458908,-7.866801,-1.549721e-06,1.458789,-7.865342,0.000978,1.851878e+09,204.0,204.0,204.0,9211.0,0.0,10.0,12.0
4,0.0,3.388784,0.238046,26.827122,26.827043,-6.389652,-15.000002,4.660606e+00,-6.389633,-14.999928,4.660628,1.851878e+09,204.0,204.0,204.0,19175.0,0.0,10.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7870,0.0,3.175550,-0.043649,10.223741,10.221073,1.410354,-0.390899,6.509816e+00,1.412256,-0.392102,6.511249,1.986094e+09,204.0,204.0,204.0,12366.0,8.7,-5.0,12.0
7871,0.0,3.093101,-0.215007,17.843813,17.846757,-3.135930,0.857685,1.668930e-06,-3.137882,0.858651,-0.001978,1.851878e+09,204.0,204.0,204.0,9780.0,8.7,-5.0,12.0
7872,0.0,2.982794,-0.269869,17.034615,17.037625,-2.825179,-1.346459,1.430511e-06,-2.827215,-1.345813,-0.002119,1.851878e+09,204.0,204.0,204.0,6318.0,8.7,-5.0,12.0
7873,0.0,3.204597,-0.318747,16.110165,16.109950,-0.089574,1.187146,-9.536743e-07,-0.089457,1.187063,0.000159,1.851878e+09,204.0,204.0,204.0,13207.0,8.7,-5.0,12.0


In [7]:
### get the leaves only
leaves = scan['object_id'] == 1986094444.0
points = np.vstack((np.array(scan['x']), np.array(scan['y']), np.array(scan['z']))).transpose()
# pcd = lad.points2pcd(points)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[leaves])
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=np.asarray(pcd.points).shape))
o3d.visualization.draw_geometries([pcd])

In [15]:
# Here we show how some of the point cloud parameters looks like.
cols = ['x','y']

plt.figure(figsize=(18, 4*3))

for num, i in enumerate(cols):
    
    plt.subplot(3,3,num+1)
    plt.hist(scan[i][leaves], 80, alpha=0.5, label=i)
    plt.legend()
    
plt.show()

plt.figure(figsize=(14, 8))
plt.scatter(scan['x'][leaves], scan['y'][leaves], s=1)

In [8]:
np.sum(leaves)

3446

In [9]:
#
plt.figure(figsize=(14, 8))

# x = np.linspace(0, 16, 3)
# y = np.array([0, 8])

# centres
x, y = [0], [0]

trees = {}
num = 0
# dx, dy = 4, 4
dx, dy = 5, 5

resdir_name = 'toy_trees'
resdir = os.path.join(datapath, resdir_name)
if not os.path.exists(resdir):
    os.makedirs(resdir)

for i in x:
    for j in y:
        
        keep = np.ones(len(scan['x']), dtype=bool)
        keep &= (scan['x'] < i+dx) & (scan['x'] > i-dx)
        keep &= (scan['y'] < j+dy) & (scan['y'] > j-dy)
        
        plt.scatter(scan['x'][leaves & keep], scan['y'][leaves & keep], s=0.5, label=num)
        
        np.save(os.path.join(resdir, 'tree_%s' %(str(num))), scan[leaves & keep])
        
        num += 1
        
plt.legend()   

# DEV ZONE...

# define the path of the data stored
mockname = 'single'
datapath = os.path.join(_data, mockname, 'toy_trees')
file = glob.glob(os.path.join(datapath, 'tree*.npy'))
print(file)
df = np.load(file[0])

In [123]:
mockname = 'single_tmp'
datapath = os.path.join(_data, mockname)

# read the numpy files
files = {}
plt.figure(figsize=(18,14))
for file in glob.glob(os.path.join(datapath, 's*.npy')):
    filename = file.split('/')[-1]
    df = np.load(file)
    # df = df[::1]
    yaw = df.T[1]
    pitch = df.T[2]
    x, y = df.T[5], df.T[6]

    
    plt.subplot(3,2,1)
    plt.title('yaw')
    #np.linspace(2.8, 3.5, 40)
    plt.hist(np.rad2deg(yaw), bins=40, histtype='step', lw=2, label=filename)
    plt.legend()
    plt.subplot(3,2,2)
    plt.title('pitch')
    #np.linspace(-0.35, 0.3, 40)
    plt.hist(np.rad2deg(pitch), bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,3)
    plt.title('x')
    plt.hist(x, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,4)
    plt.title('y')
    plt.hist(y, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,5)
    uv = lad.beam_direction(yaw, pitch, scan_inc=90)
    beam_angs = []
    for i in uv:
        beam_angs.append(lad.vecttoangle([0, 0, 1], i))
    plt.hist(np.array(beam_angs), 40, histtype='step', label='pre correction')
    plt.legend()
    plt.subplot(3,2,6)
    plt.hist(uv.T[0], bins=40, histtype='step', lw=2, label=filename+' x')
    plt.hist(uv.T[1], bins=40, histtype='step', lw=2, label=filename+' y')
    plt.hist(uv.T[2], bins=40, histtype='step', lw=2, label=filename+' z')
    plt.legend()


In [130]:
colors = ['b', 'cyan', 'orange', 'yellow']
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

for num, file in enumerate(glob.glob(os.path.join(datapath, 's1*.npy'))):
    
    filename = file.split('/')[-1]
    df = np.load(file)
    df = df[::10]
    print(df.shape)
    # yaw = df.T[1]
    # pitch = df.T[2]
    x, y, z = df.T[5], df.T[6], df.T[7]
    # sensor coordinates
    spos = os.path.join(datapath, 'scanner_pos.txt')
    scan = lad.laod_scan_pos(spos)
    id = [i.decode("utf-8") for i in scan['scan']]
    keep = np.array(id) == filename[:2]
    _, sx, sy, sz = scan[keep][0]
    print(scan[keep][0])

    ax.scatter3D(x, y, z, c='g', s=1)
    ax.scatter3D(sx, sy, sz, c='r', s=10)

    p2 = [sx, sy, sz]
    res = 2

    for i, j, k in zip(x, y, z):

        p1 = [i, j, k]

        px, py, pz = lad.line2points_vect(p1, p2, res=res)
        # ax.scatter3D(px, py, pz, c='k', s=1)
        ax.plot3D([i, sx], [j, sy], [k, sz], lw=0.1, color=colors[num])


In [9]:
# get true leag-angle-distribution
file = "../data/single_57_inv/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
# 
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()
# print(mesh.cluster_connected_triangles())
# print("Computing normal and rendering it.")
# 
print(np.asarray(mesh.triangle_normals))
print(np.asarray(mesh.triangles))
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)


[[0.37349295 0.25693007 0.89134166]
 [0.3734951  0.25692643 0.89134181]
 [0.37349183 0.25692803 0.89134272]
 ...
 [0.87693035 0.28342595 0.38815318]
 [0.87693022 0.28342695 0.38815275]
 [0.87693417 0.28341742 0.38815077]]
[[    0     1     2]
 [    0     2     3]
 [    0     3     4]
 ...
 [41138 41140 41141]
 [41142 41143 41144]
 [41142 41144 41145]]


In [6]:
# get leaf area
file = "../data/spherical_02/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
cidx, nt, area = mesh.cluster_connected_triangles()
la = np.array(area)[np.array(nt) == 4][0]
np.round(la, 4)

0.0267

In [17]:
len(area)

7144

In [11]:
set(np.round(np.array(area), 5))

{0.01333, 0.02667}

In [21]:
np.array(area)[np.array(nt) == 4][0]

0.026666766174740775